In [1]:
# Import necessary packages
from stattlepy import Stattleship
import pandas as pd
from itertools import product

In [2]:
# Transfers Slug from 'Titans vs Cardinals December 10, 2017 at  4:05pm' to 'nfl-2017-2018-ten-ari-2017-12-10-1605'
def func(s):
    """Changes Game Name to Game Slug"""
    l = s.split(' ')
    while '' in l:
        l.remove('')
    season_dict = {('September', '2015'): 'nfl-2015-2016', ('October', '2015'): 'nfl-2015-2016', ('November', '2015'): 'nfl-2015-2016', ('December', '2015'): 'nfl-2015-2016', ('January', '2016'): 'nfl-2015-2016', ('February', '2016'): 'nfl-2015-2016', ('September', '2016'): 'nfl-2016-2017', ('October', '2016'): 'nfl-2016-2017', ('November', '2016'): 'nfl-2016-2017', ('December', '2016'): 'nfl-2016-2017', ('January', '2017'): 'nfl-2016-2017', ('February', '2017'): 'nfl-2016-2017', ('September', '2017'): 'nfl-2017-2018', ('October', '2017'): 'nfl-2017-2018', ('November', '2017'): 'nfl-2017-2018', ('December', '2017'): 'nfl-2017-2018', ('January', '2018'): 'nfl-2017-2018', ('February', '2018'): 'nfl-2017-2018'}
    e1 = season_dict[l[3], l[5]]
    team_dict = {'Cardinals': 'ari', 'Falcons': 'atl', 'Ravens': 'bal', 'Bills': 'buf', 'Panthers': 'car', 'Bears': 'chi', 'Bengals': 'cin', 'Browns': 'cle', 'Cowboys': 'dal', 'Broncos': 'den', 'Lions': 'det', 'Packers': 'gb', 'Texans': 'hou', 'Colts': 'ind', 'Jaguars': 'jac', 'Chiefs': 'kc', 'Chargers': 'lac', 'Rams': 'stl', 'Dolphins': 'mia', 'Vikings': 'min', 'Patriots': 'ne', 'Saints': 'no', 'Giants': 'nyg', 'Jets': 'nyj', 'Raiders': 'oak', 'Eagles': 'phi', 'Steelers': 'pit', '49ers': 'sf', 'Seahawks': 'sea', 'Buccaneers': 'tb', 'Titans':'ten', 'Redskins': 'was'}
    e2 = team_dict[l[0]]
    e3 = team_dict[l[2]]
    month_dict = {'September': '09', 'October': '10', 'November': '11', 'December': '12', 'January': '01', 'February': '02'}
    e4 = month_dict[l[3]]
    day_dict = {'1,': '1', '2,': '2', '3,': '3', '4,': '4', '5,': '5', '6,': '6', '7,': '7', '8,': '8', '9,': '9', '10,': '10', '11,': '11', '12,': '12', '13,': '13', '14,': '14', '15,': '15', '16,': '16', '17,': '17', '18,': '18', '19,': '19', '20,': '20', '21,': '21', '22,': '22', '23,': '23', '24,': '24', '25,': '25', '26,': '26', '27,': '27', '28,': '28', '29,': '29', '30,': '30', '31,': '31'}
    e5 = day_dict[l[4]]
    time_dict = {'9:30am': '0930', '12:00pm': '1200' , '12:30pm': '1230', '1:00pm': '1300', '1:05pm': '1305', '1:25pm': '1325' , '2:25pm': '1425', '4:05pm': '1605', '4:25pm': '1625', '4:30pm': '1630', '5:30pm': '1730' , '6:25pm': '1825' , '6:30pm': '1830' , '7:10pm': '1910', '8:15pm': '2015', '8:20pm': '2020','8:25pm' : '2025', '8:30pm': '2030', '10:20pm': '2220'}
    e6 = time_dict[l[7]]
    return e1 + '-' + e2 + '-' + e3 + '-' + str(l[5]) + '-' + e4 + '-' + e5 + '-' + e6

In [3]:
# Empty list for slugs
slugs = []

# List of slugs for Query
season_slug = ['nfl-2015-2016', 'nfl-2016-2017', 'nfl-2017-2018']
team_slug = ['nfl-ari', 'nfl-atl', 'nfl-bal', 'nfl-buf', 'nfl-car', 'nfl-chi', 'nfl-cin', 'nfl-cle', 'nfl-dal', 'nfl-den', 'nfl-det', 'nfl-gb', 'nfl-hou', 'nfl-ind', 'nfl-jac', 'nfl-kc', 'nfl-lac', 'nfl-stl', 'nfl-mia', 'nfl-min', 'nfl-ne', 'nfl-no', 'nfl-nyg', 'nfl-nyj', 'nfl-oak', 'nfl-phi', 'nfl-pit', 'nfl-sf', 'nfl-sea', 'nfl-tb', 'nfl-ten', 'nfl-was']

New_query = Stattleship()
Token = New_query.set_token('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

# For Loop to go through each season for each team and add games to slugs list
for season, team in product(season_slug, team_slug):
    print(season, team)
    Output = New_query.ss_get_results(sport='football', league='nfl', ep='team_game_logs', season_id=season, team_id=team)
    slugs.append(Output)
    
# Flattens the list
flattened_list = [item for sub_list in slugs for item in sub_list]

# Adds the name of each game to a new list called log
name_list = flattened_list[0]['games'][:]
log = [record['name'] for record in name_list]

('nfl-2015-2016', 'nfl-ari')
Making Initial API Request
<Response [200]>
https://api.stattleship.com/football/nfl/team_game_logs?season_id=nfl-2015-2016&team_id=nfl-ari&page=1
Stattleship API request complete
('nfl-2015-2016', 'nfl-atl')
Making Initial API Request
<Response [200]>
https://api.stattleship.com/football/nfl/team_game_logs?season_id=nfl-2015-2016&team_id=nfl-atl&page=1
Stattleship API request complete
('nfl-2015-2016', 'nfl-bal')
Making Initial API Request
<Response [200]>
https://api.stattleship.com/football/nfl/team_game_logs?season_id=nfl-2015-2016&team_id=nfl-bal&page=1
Stattleship API request complete
('nfl-2015-2016', 'nfl-buf')
Making Initial API Request
<Response [200]>
https://api.stattleship.com/football/nfl/team_game_logs?season_id=nfl-2015-2016&team_id=nfl-buf&page=1
Stattleship API request complete
('nfl-2015-2016', 'nfl-car')
Making Initial API Request
<Response [200]>
https://api.stattleship.com/football/nfl/team_game_logs?season_id=nfl-2015-2016&team_id=nfl

In [4]:
# Create new list with all game names to go through loop to get all the game logs
games = []
for row in log:
    games.append(row)

df = pd.DataFrame()

def func_2(Output_two):
    """Gets outcomes for each game for DataFrame"""
    if Output_two[0]['team_game_logs'][0]['away_team_outcome'] == 'win':
        away_win = Output_two[0]['away_teams'][0]['nickname']
        home_loss = Output_two[0]['home_teams'][0]['nickname']
        return away_win, home_loss, 'road win'
    elif Output_two[0]['team_game_logs'][0]['away_team_outcome'] == 'loss':
        home_win = Output_two[0]['home_teams'][0]['nickname']
        away_loss = Output_two[0]['away_teams'][0]['nickname']
        return home_win, away_loss, 'road loss'
    else:
        away_tie = Output_two[0]['home_teams'][0]['nickname']
        home_tie = Output_two[0]['away_teams'][0]['nickname']
        return away_tie, home_tie, 'tie'

In [7]:
# For Loop to get the stats from each game
for name in games:
    name = func(name)
    season = name[:13]
    Output_two = New_query.ss_get_results(sport='football',league='nfl', ep='team_game_logs', season_id=season, game_id=name)
    s = pd.Series()
    winning_team, losing_team, outcome = func_2(Output_two)
    if outcome != 'tie':
        # Gets name of winning and losing teams
        s['Name of Winning Team'] = winning_team
        s['Name of Losing Team'] = losing_team
    else:
        # Gets name for both tying teams if there is a tie
        s['Name of Tying Teams'] = [winning_team, losing_team]
    s['Away Team Name'] = Output_two[0]['away_teams'][0]['nickname']
    s['Home Team Name'] = Output_two[0]['home_teams'][0]['nickname']
    # Road Team's Stats
    s['Road Win'] = Output_two[0]['team_game_logs'][0]['away_team_outcome'] == 'win'
    s['Road Loss'] = Output_two[0]['team_game_logs'][0]['team_outcome'] == 'loss'
    s['Road Tie'] = Output_two[0]['team_game_logs'][0]['team_outcome'] == 'tie'
    s['Road Wins'] = Output_two[0]['team_game_logs'][0]['games_won']
    s['Road Losses'] = Output_two[0]['team_game_logs'][0]['games_lost']
    s['Road Ties'] = Output_two[0]['team_game_logs'][0]['games_tied']
    s['Road Points'] = Output_two[0]['team_game_logs'][0]['points']
    s['Road Rushing Atts'] = Output_two[0]['team_game_logs'][0]['rushing_plays']
    s['Road Rushing Yds'] = Output_two[0]['team_game_logs'][0]['rushing_gross_yards']
    s['Road Rushing TDs'] = Output_two[0]['team_game_logs'][0]['rushing_touchdowns']
    s['Road Rushing First Downs'] = Output_two[0]['team_game_logs'][0]['rushing_first_downs']
    s['Road Passing Atts'] = Output_two[0]['team_game_logs'][0]['passing_plays_attempted']
    s['Road Passing Completions'] = Output_two[0]['team_game_logs'][0]['passing_plays_completed']
    s['Road Passing Yds'] = Output_two[0]['team_game_logs'][0]['passing_gross_yards']
    s['Road Passing TDs'] = Output_two[0]['team_game_logs'][0]['passing_touchdowns']
    s['Road Passing First Downs'] = Output_two[0]['team_game_logs'][0]['passing_first_downs']
    s['Road Total Yds'] = Output_two[0]['team_game_logs'][0]['rushing_gross_yards'] + Output_two[0]['team_game_logs'][0]['passing_gross_yards']
    s['Road Total TDs'] = Output_two[0]['team_game_logs'][0]['total_touchdowns']
    s['Road First Downs'] = Output_two[0]['team_game_logs'][0]['total_first_downs']
    s['Road Third Down Attempts'] = Output_two[0]['team_game_logs'][0]['third_downs_attempted']
    s['Road Third Down Successes'] = Output_two[0]['team_game_logs'][0]['third_downs_succeeded']
    s['Road Fourth Down Attempts'] = Output_two[0]['team_game_logs'][0]['fourth_downs_attempted']
    s['Road Fourth Down Successes'] = Output_two[0]['team_game_logs'][0]['fourth_downs_succeeded']
    s['Road Red Zone Attempts'] = Output_two[0]['team_game_logs'][0]['red_zone_scores_attempted']
    s['Road Red Zone Successes'] = Output_two[0]['team_game_logs'][0]['red_zone_scores_succeeded']
    s['Road Goal To Go Attempts'] = Output_two[0]['team_game_logs'][0]['goal_to_go_attempted']
    s['Road Goal To Go Successes'] = Output_two[0]['team_game_logs'][0]['goal_to_go_succeeded']
    s['Road Two Points Conversion Attempts'] = Output_two[0]['team_game_logs'][0]['extra_point_passes_attempted'] + Output_two[0]['team_game_logs'][0]['extra_point_rushes_attempted']
    s['Road Two Points Conversion Successes'] = Output_two[0]['team_game_logs'][0]['extra_point_passes_succeeded'] + Output_two[0]['team_game_logs'][0]['extra_point_rushes_succeeded']
    s['Road Time of Possession'] = Output_two[0]['team_game_logs'][0]['time_of_possession_secs']
    s['Road Sacks Given Up'] = Output_two[0]['team_game_logs'][0]['passing_plays_sacked']
    s['Road Sacks Yds Given Up'] = Output_two[0]['team_game_logs'][0]['passing_sacked_yards']
    s['Road INTs'] = Output_two[0]['team_game_logs'][0]['interception_returns']
    s['Road INTs Given Up'] = Output_two[0]['team_game_logs'][0]['passing_plays_intercepted']
    s['Road INT TDs'] = Output_two[0]['team_game_logs'][0]['interception_touchdowns']
    s['Road Fumbles'] = Output_two[0]['team_game_logs'][0]['fumbles']
    s['Road Fumbles Lost'] = Output_two[0]['team_game_logs'][0]['fumbles_lost']
    s['Road Fumbles Recovered'] = Output_two[0]['team_game_logs'][0]['fumbles_recovered_defense']
    s['Road Fumble TDs'] = Output_two[0]['team_game_logs'][0]['fumble_touchdowns']
    s['Road Safeties'] = Output_two[0]['team_game_logs'][0]['safeties']
    s['Road Field Goal Attempts'] = Output_two[0]['team_game_logs'][0]['field_goals_attempted']
    s['Road Made Field Goals'] = Output_two[0]['team_game_logs'][0]['field_goals_succeeded']
    s['Road Kickoffs'] = Output_two[0]['team_game_logs'][0]['kickoffs']
    s['Road Kickoff Returns'] = Output_two[0]['team_game_logs'][0]['kickoff_returns']
    s['Road Kickoff Return YDs'] = Output_two[0]['team_game_logs'][0]['kickoff_return_yards']
    s['Road Kickoff Return TDs'] = Output_two[0]['team_game_logs'][0]['kickoff_return_touchdowns']
    s['Road Touchbacks'] = Output_two[0]['team_game_logs'][0]['kickoff_touchbacks']
    s['Road Punt Returns'] = Output_two[0]['team_game_logs'][0]['punt_returns']
    s['Road Punt Return Yds'] = Output_two[0]['team_game_logs'][0]['punt_return_yards']
    s['Road Punt Return TDs'] = Output_two[0]['team_game_logs'][0]['punt_return_touchdowns']
    s['Road Punts'] = Output_two[0]['team_game_logs'][0]['punting_plays']
    s['Road Punting Net Yards'] = Output_two[0]['team_game_logs'][0]['punting_net_yards_avg']
    s['Road Penalties'] = Output_two[0]['team_game_logs'][0]['penalties']
    s['Road Penalty Yards'] = Output_two[0]['team_game_logs'][0]['penalty_yards']
    s['Road Penalty First Downs'] = Output_two[0]['team_game_logs'][0]['penalty_first_downs']
    # Home Team's Stats
    s['Home Win'] = Output_two[0]['team_game_logs'][1]['away_team_outcome'] == 'win'
    s['Home Loss'] = Output_two[0]['team_game_logs'][1]['team_outcome'] == 'loss'
    s['Home Tie'] = Output_two[0]['team_game_logs'][1]['team_outcome'] == 'tie'
    s['Home Wins'] = Output_two[0]['team_game_logs'][1]['games_won']
    s['Home Losses'] = Output_two[0]['team_game_logs'][1]['games_lost']
    s['Home Ties'] = Output_two[0]['team_game_logs'][1]['games_tied']
    s['Home Points'] = Output_two[0]['team_game_logs'][1]['points']
    s['Home Rushing Atts'] = Output_two[0]['team_game_logs'][1]['rushing_plays']
    s['Home Rushing Yds'] = Output_two[0]['team_game_logs'][1]['rushing_gross_yards']
    s['Home Rushing TDs'] = Output_two[0]['team_game_logs'][1]['rushing_touchdowns']
    s['Home Rushing First Downs'] = Output_two[0]['team_game_logs'][1]['rushing_first_downs']
    s['Home Passing Atts'] = Output_two[0]['team_game_logs'][1]['passing_plays_attempted']
    s['Home Passing Completions'] = Output_two[0]['team_game_logs'][1]['passing_plays_completed']
    s['Home Passing Yds'] = Output_two[0]['team_game_logs'][1]['passing_gross_yards']
    s['Home Passing TDs'] = Output_two[0]['team_game_logs'][1]['passing_touchdowns']
    s['Home Passing First Downs'] = Output_two[0]['team_game_logs'][1]['passing_first_downs']
    s['Home Total Yds'] = Output_two[0]['team_game_logs'][1]['rushing_gross_yards'] + Output_two[0]['team_game_logs'][1]['passing_gross_yards']
    s['Home Total TDs'] = Output_two[0]['team_game_logs'][1]['total_touchdowns']
    s['Home First Downs'] = Output_two[0]['team_game_logs'][1]['total_first_downs']
    s['Home Third Down Attempts'] = Output_two[0]['team_game_logs'][1]['third_downs_attempted']
    s['Home Third Down Successes'] = Output_two[0]['team_game_logs'][1]['third_downs_succeeded']
    s['Home Fourth Down Attempts'] = Output_two[0]['team_game_logs'][1]['fourth_downs_attempted']
    s['Home Fourth Down Successes'] = Output_two[0]['team_game_logs'][1]['fourth_downs_succeeded']
    s['Home Red Zone Attempts'] = Output_two[0]['team_game_logs'][1]['red_zone_scores_attempted']
    s['Home Red Zone Successes'] = Output_two[0]['team_game_logs'][1]['red_zone_scores_succeeded']
    s['Home Goal To Go Attempts'] = Output_two[0]['team_game_logs'][1]['goal_to_go_attempted']
    s['Home Goal To Go Successes'] = Output_two[0]['team_game_logs'][1]['goal_to_go_succeeded']
    s['Home Two Points Conversion Attempts'] = Output_two[0]['team_game_logs'][1]['extra_point_passes_attempted'] + Output_two[0]['team_game_logs'][1]['extra_point_rushes_attempted']
    s['Home Two Points Conversion Successes'] = Output_two[0]['team_game_logs'][1]['extra_point_passes_succeeded'] + Output_two[0]['team_game_logs'][1]['extra_point_rushes_succeeded']
    s['Home Time of Possession'] = Output_two[0]['team_game_logs'][1]['time_of_possession_secs']
    s['Home Sacks Given Up'] = Output_two[0]['team_game_logs'][1]['passing_plays_sacked']
    s['Home Sacks Yds Given Up'] = Output_two[0]['team_game_logs'][1]['passing_sacked_yards']
    s['Home INTs'] = Output_two[0]['team_game_logs'][1]['interception_returns']
    s['Home INTs Given Up'] = Output_two[0]['team_game_logs'][1]['passing_plays_intercepted']
    s['Home INT TDs'] = Output_two[0]['team_game_logs'][1]['interception_touchdowns']
    s['Home Fumbles'] = Output_two[0]['team_game_logs'][1]['fumbles']
    s['Home Fumbles Lost'] = Output_two[0]['team_game_logs'][1]['fumbles_lost']
    s['Home Fumbles Recovered'] = Output_two[0]['team_game_logs'][1]['fumbles_recovered_defense']
    s['Home Fumble TDs'] = Output_two[0]['team_game_logs'][1]['fumble_touchdowns']
    s['Home Safeties'] = Output_two[0]['team_game_logs'][1]['safeties']
    s['Home Field Goal Attempts'] = Output_two[0]['team_game_logs'][1]['field_goals_attempted']
    s['Home Made Field Goals'] = Output_two[0]['team_game_logs'][1]['field_goals_succeeded']
    s['Home Kickoffs'] = Output_two[0]['team_game_logs'][1]['kickoffs']
    s['Home Kickoff Returns'] = Output_two[0]['team_game_logs'][1]['kickoff_returns']
    s['Home Kickoff Return YDs'] = Output_two[0]['team_game_logs'][1]['kickoff_return_yards']
    s['Home Kickoff Return TDs'] = Output_two[0]['team_game_logs'][1]['kickoff_return_touchdowns']
    s['Home Touchbacks'] = Output_two[0]['team_game_logs'][1]['kickoff_touchbacks']
    s['Home Punt Returns'] = Output_two[0]['team_game_logs'][1]['punt_returns']
    s['Home Punt Return Yds'] = Output_two[0]['team_game_logs'][1]['punt_return_yards']
    s['Home Punt Return TDs'] = Output_two[0]['team_game_logs'][1]['punt_return_touchdowns']
    s['Home Punts'] = Output_two[0]['team_game_logs'][1]['punting_plays']
    s['Home Punting Net Yards'] = Output_two[0]['team_game_logs'][1]['punting_net_yards_avg']
    s['Home Penalties'] = Output_two[0]['team_game_logs'][1]['penalties']
    s['Home Penalty Yards'] = Output_two[0]['team_game_logs'][1]['penalty_yards']
    s['Home Penalty First Downs'] = Output_two[0]['team_game_logs'][1]['penalty_first_downs']
    df = pd.concat([df, s.to_frame()])

df.head(120)

Making Initial API Request
<Response [200]>
https://api.stattleship.com/football/nfl/team_game_logs?season_id=nfl-2015-2016&game_id=nfl-2015-2016-ari-phi-2015-12-20-2030&page=1
Stattleship API request complete
Making Initial API Request
<Response [200]>
https://api.stattleship.com/football/nfl/team_game_logs?season_id=nfl-2015-2016&game_id=nfl-2015-2016-ari-sf-2015-11-29-1305&page=1
Stattleship API request complete
Making Initial API Request
<Response [200]>
https://api.stattleship.com/football/nfl/team_game_logs?season_id=nfl-2015-2016&game_id=nfl-2015-2016-gb-ari-2015-12-27-1425&page=1
Stattleship API request complete
Making Initial API Request
<Response [200]>
https://api.stattleship.com/football/nfl/team_game_logs?season_id=nfl-2015-2016&game_id=nfl-2015-2016-cin-ari-2015-11-22-1830&page=1
Stattleship API request complete
Making Initial API Request
<Response [200]>
https://api.stattleship.com/football/nfl/team_game_logs?season_id=nfl-2015-2016&game_id=nfl-2015-2016-sea-ari-2016-01-

,0
Name of Winning Team,Cardinals
Name of Losing Team,Eagles
Away Team Name,Cardinals
Home Team Name,Eagles
Road Win,True
Road Loss,False
Road Tie,False
Road Wins,1
Road Losses,0
Road Ties,0
